[Node 51: SQLite mit sqlite3](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node51.html)

Navigation:

**Next:** [Aufgaben](node52.ipynb) **Up:** [Datenformate – I](node48.ipynb) **Previous:** [DB-API 2.0 SQL-Anbindungen](node50.ipynb)

##  SQLite mit sqlite3

SQLite ist im Gegensatz zu anderen SQL-Datenbanksystemen kein Server sondern eine in    <font color=#0000ff> **C**</font>  geschriebene Bibliothek. Es werden SQL-Abfragen und -Anweisungen unterstützt. Dabei wird eine normale index-sequentielle Datei benutzt. SQLite hat geringe Anforderungen  an CPU und Speicher.    
 
Zu Anfang müssen wir ein Schema, also die Datenbankstruktur, festlegen:

```bash
laptop:~/python/kurs09$ sqlite3 /tmp/blogdb.sqlite3
SQLite version 3.6.10
Enter ".help" for instructions
Enter SQL statements terminated with a ";"
sqlite> CREATE table comments (
id INTEGER PRIMARY KEY AUTOINCREMENT,
subject TEXT,
author TEXT,
text TEXT
);
sqlite> .quit
```

 Wir können sie auch aus einer Datei einlesen:

```bash
laptop:~/python/kurs09$ rm /tmp/blogdb.sqlite3
laptop:~/python/kurs09$ sqlite3 /tmp/blogdb.sqlite3 < blogdb.schema
```

Nun kann man per   <font color=#0000e6> ``INSERT``</font>-Anweisungen Tabellen manuell füllen oder per   <font color=#0000e6> ``SELECT``</font>  Tabellen abfragen.   Wichtige Befehle im sqlite-Kommandozeileninterpreter:  
*  <font color=#0000e6> ``.help``</font> 
*  <font color=#0000e6> ``.schema``</font> : Zeigt Datenbankschema an 
*  <font color=#0000e6> ``.dump``</font> : Zeigt Schema und Daten einer Datenbank an (Backup)  

Werte in Datenbank einfügen:
```bash
sqlite> INSERT INTO comments VALUES(
1,
'The Python Blog',
'Max Mustermann',
'How are you ?'
);
```
```bash
sqlite> INSERT INTO comments VALUES(
2,
'The Perl Blog',
'Marie Mustermann',
'How are you ?'
);
```

 Werte abfragen:

```bash
sqlite> SELECT subject FROM comments ORDER by author;
The Perl Blog
The Python Blog
sqlite> SELECT subject,text FROM comments ORDER by author;
The Perl Blog|How are you ?
The Python Blog|How are you ?
```

### SQlite in Python

Verbinden der SQlite-Datenbank in Python mit   <font color=#0000e6> ``sqlite3``</font>-Modul (nicht autocommit-Modus):

In [ ]:
import sqlite3
conn = sqlite3.connect('data/blogdb.sqlite3', isolation_level='DEFERRED')
conn

 Wichtige Methoden:  
*  <font color=#0000e6> ``close``</font> : Schließt Datenbank-Verdindung. Ist autocommit  ausgeschaltet, werden Transaktionen ohne   <font color=#0000e6> ``commit``</font>  nicht ausgeführt. 
*  <font color=#0000e6> ``commit``</font> : schließt offene Transaktionen ab und speichert diese  in der Datenbank 
*  <font color=#0000e6> ``rollback``</font> : Transaktion abbrechen. 
*  <font color=#0000e6> ``cursor``</font> :   <font color=#0000e6> ``Cursor``</font> -Objekt zur Datenbankabfrage, Daten eintragen, verändern, löschen.

In [ ]:
curs = conn.cursor()

 Wichtige   <font color=#0000e6> ``cursor``</font>  Methoden:  
*  <font color=#0000e6> ``close, execute, executemany, fetchone, fetchall``</font>  

Daten in Datenbank eintragen:

In [ ]:
curs.execute('INSERT INTO comments VALUES (? , ?, ?, ?)', (None, 'a subject','an author', 'a text'))

In [ ]:
curs.rowcount

In [ ]:
curs.execute('INSERT INTO comments VALUES (? , ?, ?, ?)', (None, 'another subject','another author', 'another text'))

In [ ]:
curs.rowcount

In [ ]:
conn.commit()

 Aus Sicherheitsgründen (SQL injection vulnerability) sollte immer mit  Platzhaltern '?' gearbeitet werden.

In [ ]:
curs.execute('SELECT * FROM comments ORDER BY id')

In [ ]:
curs.rowcount

(`rowcount` liefert bei `SELECT` allerdings [immer -1](https://stackoverflow.com/questions/839069/cursor-rowcount-always-1-in-sqlite3-in-python3k) zurück, lässt sich also nicht verwenden, um die Anzahl der gefundenen Ergebnisse abzufragen.)

In [ ]:
curs.fetchone()

In [ ]:
curs.fetchone()

In [ ]:
curs.fetchone()

In [ ]:
curs.fetchone()

In [ ]:
curs.execute('SELECT * FROM comments ORDER BY id')

In [ ]:
result=curs.fetchone()
while result is not None:
    print (result)
    result = curs.fetchone()

### Einschub: SQL Vulnerability

SQL-Anweisungen sind anfällig gegen Einschleusen weiterer  (potentiell schädlicher) SQL-Kommandos, wie in diesem [XKCD](https://xkcd.com/)-Comic [demonstriert](https://www.explainxkcd.com/wiki/index.php/327:_Exploits_of_a_Mom):

[![Image exploits_of_a_mom](https://imgs.xkcd.com/comics/exploits_of_a_mom.png "Her daughter is named Help I&#39;m trapped in a driver&#39;s license factory.")](https://xkcd.com/327/)

Verwendung von expliziten Platzhaltern (`?` bei unseren `INSERT`s oben) sichert dagegen ab.

Aus Speichergründen ist es besser, <font color=#0000e6> ``fetchone``</font>  oder   <font color=#0000e6> ``fetchmany``</font>  zu  verwenden.   
 
Datensätze verändern oder löschen:

In [ ]:
curs.execute('UPDATE comments SET author=? WHERE id=?', ('me',2))

In [ ]:
curs.rowcount

In [ ]:
conn.commit()
curs.execute('SELECT * FROM comments WHERE id=?', (2,))

In [ ]:
curs.fetchone()

In [ ]:
curs.execute('DELETE FROM comments WHERE id<?', (3,))

In [ ]:
curs.rowcount

In [ ]:
conn.rollback()
curs.execute('SELECT count(*) FROM comments')

In [ ]:
curs.fetchone()

In [ ]:
curs.execute('SELECT * FROM comments ORDER BY id')

In [ ]:
curs.fetchall()

In [ ]:
conn.close()

Soweit nur absolute Grundlagen der Datenbanknutzung. 
<font color=#0000ff> **Relationale Datenbanken**</font>  bieten viele weitere Features und spielen ihre Stärken insbesondere beim Verknüpfen verschiedener Tabellen an, was aber über den Kurs hier hinausgeht.

Gute Einführung:  [**Software Carpentry: Databases and SQL**](http://swcarpentry.github.io/sql-novice-survey/index.html)